In [6]:
# from git import Repo
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [2]:
repo_path = "/mnt/nvme1n1/Aamir/CurePilot/WebServices/HISApi"

In [4]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".cs"],
    exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=500),
)
documents = loader.load()
len(documents)

229

In [5]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

302

In [15]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=(), api_key=""))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [19]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    api_key="",
)
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [21]:
question = "what are the methods available inside DrugTypesController"
result = qa(question)
result["answer"]

'Inside the `DrugTypesController` class, the following methods are available:\n\n1. `UpdateDrugType`: This method is a POST method with the route `[Route(RouteConstants.UpdateDrugType)]`. It takes a parameter of type `ViewDrugNameDto` and returns an `IHttpActionResult`.\n\n2. `DetermineDrugTypeByRpid`: This method is a GET method with the route `[Route(RouteConstants.DetermineTypeByRpid)]`. It takes an `int` parameter `rpid` and returns an `IHttpActionResult`.\n\nPlease note that these are the methods available based on the given context. There might be additional methods in the class that are not shown in the provided code.'